In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_766968/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [5]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse_sup", ascending=True, ignore_index=True)
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
25,model_18_9_20,0.999450,0.597078,0.998290,0.998360,0.998326,0.000222,0.239192,0.000845,0.000616,0.000730,0.004153,0.014895,1.000433,0.015014,584.826956,1390.331018,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
26,model_18_9_21,0.999450,0.597083,0.998288,0.998357,0.998324,0.000222,0.239189,0.000846,0.000617,0.000731,0.004140,0.014895,1.000433,0.015015,584.826934,1390.330996,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
27,model_18_9_22,0.999450,0.597088,0.998287,0.998354,0.998322,0.000222,0.239186,0.000847,0.000618,0.000732,0.004128,0.014896,1.000433,0.015015,584.826732,1390.330793,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
28,model_18_9_19,0.999450,0.597071,0.998292,0.998363,0.998329,0.000222,0.239196,0.000844,0.000614,0.000729,0.004167,0.014896,1.000433,0.015016,584.826599,1390.330661,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
29,model_18_9_23,0.999449,0.597092,0.998285,0.998352,0.998320,0.000222,0.239184,0.000848,0.000619,0.000733,0.004117,0.014898,1.000433,0.015017,584.826237,1390.330299,"Hidden Size=[15, 14], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,model_7_9_1,0.997056,0.640463,0.997069,0.997027,0.997064,0.001083,0.213436,0.001246,0.001104,0.001175,0.016380,0.032909,1.000225,0.033173,3531.656064,8520.675939,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
612,model_7_9_0,0.997016,0.640519,0.997066,0.997032,0.997064,0.001098,0.213403,0.001248,0.001102,0.001175,0.016457,0.033131,1.000228,0.033397,3531.629146,8520.649020,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
622,model_19_8_2,0.996963,0.651465,0.998138,0.999070,0.998790,0.001224,0.206905,0.000531,0.000450,0.000491,0.007736,0.034988,1.002387,0.035269,581.410987,1386.915048,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
627,model_19_8_1,0.996926,0.651511,0.998245,0.999101,0.998846,0.001239,0.206878,0.000501,0.000435,0.000468,0.007762,0.035202,1.002417,0.035484,581.386646,1386.890707,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
